# Causal Inference - Homework 4
Nitay Suissa 209446640

Matan Solomon 322853334

## Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression, Ridge
from sklearn.neighbors import NearestNeighbors

## Process data

In [2]:
data1 = pd.read_csv("data1.csv", index_col="Unnamed: 0")
data1 = pd.get_dummies(data1)
data2 = pd.read_csv("data2.csv", index_col="Unnamed: 0")
data2 = pd.get_dummies(data2)
data1.head(3)

,x_1,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,...,x_21_L,x_21_M,x_21_N,x_21_O,x_21_P,x_24_A,x_24_B,x_24_C,x_24_D,x_24_E
1,29,1.0,7.0,60,85,0,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
2,27,0.0,0.0,64,178,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
3,27,0.0,0.0,60,102,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [3]:
def calc_props(df):
    x = df[df.columns[~df.columns.isin(['T','Y'])]]
    t = df['T'].values
    log_reg = LogisticRegression(max_iter=10000).fit(x, t)
    probs = log_reg.predict_proba(x)[:,1]
    df['propensity'] = probs
    df['weighted'] = probs / (1-probs)
    return df

In [4]:
data1 = calc_props(data1)
data2 = calc_props(data2)
data1.head(3)

C:\Users\nitay\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,x_1,x_3,x_4,x_5,x_6,x_7,x_8,x_9,x_10,x_11,...,x_21_N,x_21_O,x_21_P,x_24_A,x_24_B,x_24_C,x_24_D,x_24_E,propensity,weighted
1,29,1.0,7.0,60,85,0,0,1,0,0,...,0,0,0,0,1,0,0,0,0.649703,1.854723
2,27,0.0,0.0,64,178,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.564481,1.296109
3,27,0.0,0.0,60,102,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0.676802,2.094078


## IPW

In [5]:
def IPW(df):
    t1 = df[df['T']==1]['Y'].mean()
    masked = df[df['T']==0]
    t2 = (masked['Y'] * masked['weighted']).sum() / masked['weighted'].sum()
    return t1-t2

In [6]:
IPW_1 = IPW(data1)
print(f"ATT for data1: {IPW_1}")

IPW_2 = IPW(data2)
print(f"ATT for data1: {IPW_1}")

ATT for data1: 3.7897448036779773
ATT for data1: 3.7897448036779773


## S-Learner

In [7]:
def S_learner(df, model):
    clean_df = df[df.columns[~df.columns.isin(['Y', 'propensity', 'weighted'])]]
    model.fit(clean_df, df['Y'])
    masked_df = clean_df[clean_df['T'] == 1]
    t1 = model.predict(masked_df)
    copy = masked_df.copy()
    copy['T'] = 0
    t2 = model.predict(copy)
    return (t1-t2).mean()

In [8]:
model1 = Ridge(alpha=0.1)
S_1 = S_learner(data1, model1)
print(f"ATT for data1: {S_1}")

model2 = Ridge(alpha=0.1)
S_2 = S_learner(data2, model2)
print(f"ATT for data1: {S_2}")

ATT for data1: 3.4452575500800084
ATT for data1: 3.125559533557059


## T-Learner

In [9]:
def T_learner(df, model0, model1):
    clean_df = df[df.columns[~df.columns.isin(['propensity', 'weighted'])]]
    df_1 = clean_df[clean_df['T'] == 1]
    x_1 = df_1[df_1.columns[~df_1.columns.isin(['Y', 'T'])]]
    y_1 = df_1['Y']
    df_0 = clean_df[clean_df['T'] == 0]
    x_0 = df_0[df_0.columns[~df_0.columns.isin(['Y', 'T'])]]
    y_0 = df_0['Y']

    model0.fit(x_0, y_0)
    model1.fit(x_1, y_1)
    p_0 = model0.predict(x_1)
    p_1 = model1.predict(x_1)

    return (p_1-p_0).mean()

In [10]:
model1_0, model1_1 = Ridge(alpha=0.1), Ridge(alpha=0.1)
T_1 = T_learner(data1, model1_0, model1_1)
print(f"ATT for data1: {T_1}")

model2_0, model2_1 = Ridge(alpha=0.1), Ridge(alpha=0.1)
T_2 = T_learner(data2, model2_0, model2_1)
print(f"ATT for data2: {T_2}")

ATT for data1: 3.7715680072540203
ATT for data2: 3.475670248956973


## Matching

In [11]:
def matching(df):
    clean_df = df[df.columns[~df.columns.isin(['propensity', 'weighted'])]]
    df_1 = clean_df[clean_df['T'] == 1]
    x_1 = df_1[df_1.columns[~df_1.columns.isin(['Y', 'T'])]]
    y_1 = df_1['Y']
    df_0 = clean_df[clean_df['T'] == 0]
    x_0 = df_0[df_0.columns[~df_0.columns.isin(['Y', 'T'])]]
    y_0 = df_0['Y']

    knn = NearestNeighbors(n_neighbors=1).fit(x_0)
    nei = knn.kneighbors(x_1)
    return (y_1 - y_0.iloc[nei[1].squeeze()].values).mean()

In [12]:
matching_1 = matching(data1)
print(f"ATT for data1: {matching_1}")

matching_2 = matching(data2)
print(f"ATT for data2: {matching_2}")

ATT for data1: 3.8985898906287977
ATT for data2: 3.220905396470962


## Competition

In [13]:
model1_0, model1_1 = Ridge(alpha=0.1), Ridge(alpha=0.1)
competetion_T_1 = T_learner(data1, model1_0, model1_1)
print(f"ATT for data1: {T_1}")

model2_0, model2_1 = Ridge(alpha=0.1), Ridge(alpha=0.1)
competetion_T_2 = T_learner(data2, model2_0, model2_1)
print(f"ATT for data2: {T_2}")

ATT for data1: 3.7715680072540203
ATT for data2: 3.475670248956973


## CSV

In [14]:
ATT_results = pd.DataFrame({'Type':list(range(1,6)), 'data1':[IPW_1, S_1, T_1, matching_1, competetion_T_1], 'data2':[IPW_2, S_2, T_2, matching_2, competetion_T_2]})
ATT_results

,Type,data1,data2
0,1,3.789745,3.477935
1,2,3.445258,3.125560
2,3,3.771568,3.475670
3,4,3.898590,3.220905
4,5,3.771568,3.475670


In [15]:
ATT_results.to_csv("ATT_results.csv", index=False)

In [16]:
models_propensity = pd.concat([data1[['propensity']].T, data2[['propensity']].T])
models_propensity.index = ['data1', 'data2']
models_propensity

,1,2,3,4,5,6,7,8,9,10,...,4793,4794,4795,4796,4797,4798,4799,4800,4801,4802
data1,0.649703,0.564481,0.676802,0.513441,0.609977,0.611194,0.613625,0.625979,0.671060,0.597576,...,0.644492,0.547394,0.512117,0.457145,0.434804,0.589753,0.600362,0.561732,0.418720,0.622628
data2,0.805733,0.592272,0.891983,0.291236,0.444368,0.465433,0.242084,0.798174,0.590633,0.713075,...,0.525013,0.344241,0.294394,0.297503,0.578937,0.338642,0.524295,0.516822,0.609863,0.446281


In [17]:
models_propensity.to_csv("models_propensity.csv")